In [9]:
bucket = 'bucket-mldemo-dev-20200327141354666900000003'
 
# Define IAM role
import re


In [ ]:
import boto3
boto3_session=boto3.Session(profile_name='ml-lab')

In [10]:
from sagemaker.session import Session
from sagemaker import get_execution_role

sagemaker_session=Session(boto3_session)

try:
    role = get_execution_role()
except:
    iam=boto3_session.client('iam')
    response=iam.get_role(
        RoleName='mldemo-dev-Sagemaker-Execution-Role'
    )
    role=response['Role']['Arn']
role

'arn:aws:iam::814948925568:role/mldemo-dev-Sagemaker-Execution-Role'

In [11]:
output_location = 's3://{}/output'.format(bucket)
output_location

's3://bucket-mldemo-dev-20200327141354666900000003/output'

In [68]:
from sagemaker.predictor import RealTimePredictor
linear_predictor=RealTimePredictor('mldemo-dev',sagemaker_session)

In [69]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [47]:
from s3fs.core import S3FileSystem
s3fs = S3FileSystem(anon=False,session=boto3_session)

import pandas as pd
df=pd.read_csv(s3fs.open('s3://bucket-mldemo-dev-20200327141354666900000003/transformed/all_data.csv'),header=None)
# evaluation_data_df.loc[0].tolist()[0:72]
df

,0,1,2,3,4,5,6,7,8,9,...,596,597,598,599,600,601,602,603,604,605
0,-18.0,18.0,119.05,257.16,-55.54,-0.020,0.035,0.036667,-0.433333,-0.76,...,-14.300000,-14.090000,-14.040000,-14.390000,-14.8500,-15.193333,-15.300000,-15.200000,-15.300000,0
1,-18.0,18.0,119.09,257.20,-55.50,0.020,0.075,0.076667,-0.393333,-0.72,...,-14.260000,-14.050000,-14.000000,-14.350000,-14.8100,-15.153333,-15.260000,-15.160000,-15.260000,0
2,-18.0,18.0,118.94,257.05,-55.65,-0.130,-0.075,-0.073333,-0.543333,-0.87,...,-14.410000,-14.200000,-14.150000,-14.500000,-14.9600,-15.303333,-15.410000,-15.310000,-15.410000,0
3,-18.0,18.0,119.09,257.20,-55.50,0.020,0.075,0.076667,-0.393333,-0.72,...,-14.260000,-14.050000,-14.000000,-14.350000,-14.8100,-15.153333,-15.260000,-15.160000,-15.260000,0
4,-18.0,18.0,118.94,257.05,-55.65,-0.130,-0.075,-0.073333,-0.543333,-0.87,...,-14.410000,-14.200000,-14.150000,-14.500000,-14.9600,-15.303333,-15.410000,-15.310000,-15.410000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,-18.0,18.0,95.34,233.45,-79.25,2.790,2.110,1.045000,0.680000,0.71,...,1.090000,0.953333,0.827500,0.736667,0.6200,0.532500,0.480000,0.360000,0.303333,0
10337,-18.0,18.0,95.38,233.49,-79.21,2.150,1.085,0.720000,0.750000,1.08,...,0.993333,0.867500,0.776667,0.660000,0.5725,0.520000,0.400000,0.343333,0.040000,0
10338,-18.0,18.0,95.13,233.24,-79.46,1.900,0.835,0.470000,0.500000,0.83,...,0.743333,0.617500,0.526667,0.410000,0.3225,0.270000,0.150000,0.093333,-0.210000,0
10339,-18.0,18.0,95.28,233.39,-79.31,2.050,0.985,0.620000,0.650000,0.98,...,0.893333,0.767500,0.676667,0.560000,0.4725,0.420000,0.300000,0.243333,-0.060000,0


In [50]:
evaluation_data_df=df[round(len(df)*0.85+1):].copy().reset_index()
evaluation_data_df

,index,0,1,2,3,4,5,6,7,8,...,596,597,598,599,600,601,602,603,604,605
0,8791,-18.0,18.0,90.20,228.31,-84.39,-26.930,-27.300,-27.550,-27.530000,...,0.940000,0.780000,0.740000,0.360000,0.1800,0.1600,0.150000,0.030000,0.010000,0
1,8792,-18.0,18.0,90.05,228.16,-84.54,-27.450,-27.700,-27.680,-27.700000,...,0.630000,0.590000,0.210000,0.030000,0.0100,0.0000,-0.120000,-0.140000,-0.150000,0
2,8793,-18.0,18.0,91.64,229.75,-82.95,-26.110,-26.090,-26.110,-26.040000,...,2.180000,1.800000,1.620000,1.600000,1.5900,1.4700,1.450000,1.440000,1.590000,0
3,8794,-18.0,18.0,91.61,229.72,-82.98,-26.140,-26.120,-26.140,-26.070000,...,2.150000,1.770000,1.590000,1.570000,1.5600,1.4400,1.420000,1.410000,1.560000,0
4,8795,-18.0,18.0,91.66,229.77,-82.93,-26.070,-26.090,-26.020,-26.903333,...,1.820000,1.640000,1.620000,1.610000,1.4900,1.4700,1.460000,1.610000,0.035000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,10336,-18.0,18.0,95.34,233.45,-79.25,2.790,2.110,1.045,0.680000,...,1.090000,0.953333,0.827500,0.736667,0.6200,0.5325,0.480000,0.360000,0.303333,0
1546,10337,-18.0,18.0,95.38,233.49,-79.21,2.150,1.085,0.720,0.750000,...,0.993333,0.867500,0.776667,0.660000,0.5725,0.5200,0.400000,0.343333,0.040000,0
1547,10338,-18.0,18.0,95.13,233.24,-79.46,1.900,0.835,0.470,0.500000,...,0.743333,0.617500,0.526667,0.410000,0.3225,0.2700,0.150000,0.093333,-0.210000,0
1548,10339,-18.0,18.0,95.28,233.39,-79.31,2.050,0.985,0.620,0.650000,...,0.893333,0.767500,0.676667,0.560000,0.4725,0.4200,0.300000,0.243333,-0.060000,0


In [70]:
data=','.join([str(x) for x in evaluation_data_df.loc[0].tolist()[0:605]])
linear_predictor.predict(data)

{'predictions': [{'score': 0.1967127025127411, 'predicted_label': 0.0}]}

In [71]:
def eval_func(row):
    csv=','.join(row.tolist()[0:605])
    result=linear_predictor.predict(csv)
    return result
results=evaluation_data_df.astype(str).apply(eval_func,axis=1)

In [72]:
results[0]

{'predictions': [{'score': 0.1967127025127411, 'predicted_label': 0.0}]}

In [73]:
predicted_labels=[1 if i['predictions'][0]['predicted_label']==1.0 else 0 for i in results]

In [74]:
score=[i['predictions'][0]['score'] for i in results]

In [75]:
evaluation_data_df['predicted_labels']=predicted_labels
evaluation_data_df['score']=score
evaluation_data_df

,index,0,1,2,3,4,5,6,7,8,...,598,599,600,601,602,603,604,605,predicted_labels,score
0,8791,-18.0,18.0,90.20,228.31,-84.39,-26.930,-27.300,-27.550,-27.530000,...,0.740000,0.360000,0.1800,0.1600,0.150000,0.030000,0.010000,0,0,0.196713
1,8792,-18.0,18.0,90.05,228.16,-84.54,-27.450,-27.700,-27.680,-27.700000,...,0.210000,0.030000,0.0100,0.0000,-0.120000,-0.140000,-0.150000,0,0,0.191571
2,8793,-18.0,18.0,91.64,229.75,-82.95,-26.110,-26.090,-26.110,-26.040000,...,1.620000,1.600000,1.5900,1.4700,1.450000,1.440000,1.590000,0,0,0.225435
3,8794,-18.0,18.0,91.61,229.72,-82.98,-26.140,-26.120,-26.140,-26.070000,...,1.590000,1.570000,1.5600,1.4400,1.420000,1.410000,1.560000,0,0,0.224716
4,8795,-18.0,18.0,91.66,229.77,-82.93,-26.070,-26.090,-26.020,-26.903333,...,1.620000,1.610000,1.4900,1.4700,1.460000,1.610000,0.035000,0,0,0.228285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,10336,-18.0,18.0,95.34,233.45,-79.25,2.790,2.110,1.045,0.680000,...,0.827500,0.736667,0.6200,0.5325,0.480000,0.360000,0.303333,0,0,0.593202
1546,10337,-18.0,18.0,95.38,233.49,-79.21,2.150,1.085,0.720,0.750000,...,0.776667,0.660000,0.5725,0.5200,0.400000,0.343333,0.040000,0,0,0.599678
1547,10338,-18.0,18.0,95.13,233.24,-79.46,1.900,0.835,0.470,0.500000,...,0.526667,0.410000,0.3225,0.2700,0.150000,0.093333,-0.210000,0,0,0.591395
1548,10339,-18.0,18.0,95.28,233.39,-79.31,2.050,0.985,0.620,0.650000,...,0.676667,0.560000,0.4725,0.4200,0.300000,0.243333,-0.060000,0,0,0.596373


In [76]:
evaluation_data_df[evaluation_data_df['predicted_labels']==1]

,index,0,1,2,3,4,5,6,7,8,...,598,599,600,601,602,603,604,605,predicted_labels,score
119,8910,-18.0,18.0,91.74,229.85,-82.85,-22.950,-22.910,-22.6300,-22.9700,...,2.535,2.165,2.610000,1.786667,1.920,1.930,1.8400,0,1,0.642128
121,8912,-18.0,18.0,90.60,228.71,-83.99,-24.050,-23.770,-24.1100,-24.0250,...,1.025,1.470,0.646667,0.780000,0.790,0.700,-0.2000,0,1,0.624349
122,8913,-18.0,18.0,90.59,228.70,-84.00,-24.060,-23.780,-24.1200,-24.0350,...,1.015,1.460,0.636667,0.770000,0.780,0.690,-0.2100,0,1,0.624027
123,8914,-18.0,18.0,90.61,228.72,-83.98,-24.040,-23.760,-24.1000,-24.0150,...,1.035,1.480,0.656667,0.790000,0.800,0.710,-0.1900,0,1,0.624673
124,8915,-18.0,18.0,92.22,230.33,-82.37,-22.430,-22.150,-22.4900,-22.4050,...,2.645,3.090,2.266667,2.400000,2.410,2.320,1.4200,0,1,0.675010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,9849,-18.0,18.0,92.72,230.83,-81.87,-1.545,-1.620,-1.6350,-1.7175,...,1.750,1.740,1.790000,1.915000,1.770,1.285,0.9150,0,1,0.624972
1059,9850,-18.0,18.0,92.66,230.77,-81.93,-1.605,-1.680,-1.6950,-1.7775,...,1.690,1.680,1.730000,1.855000,1.710,1.225,0.8550,0,1,0.623036
1060,9851,-18.0,18.0,92.65,230.76,-81.94,-1.690,-1.705,-1.7875,-3.3150,...,1.670,1.720,1.845000,1.700000,1.215,0.845,0.0125,0,1,0.628415
1061,9852,-18.0,18.0,92.59,230.70,-82.00,-1.750,-1.765,-1.8475,-3.3750,...,1.610,1.660,1.785000,1.640000,1.155,0.785,-0.0475,0,1,0.626487


In [94]:
most_certain=evaluation_data_df.query('score>=0.75')
most_certain.sort_values('score',ascending=False)

,index,0,1,2,3,4,5,6,7,8,...,598,599,600,601,602,603,604,605,predicted_labels,score
231,9022,-18.0,18.0,96.30,234.41,-78.29,-22.133333,-22.095000,-22.210,-22.2200,...,2.400000,2.390000,2.195000,1.506667,1.136667,0.380000,0.185,1,1,0.849549
232,9023,-18.0,18.0,96.32,234.43,-78.27,-22.075000,-22.190000,-22.200,-21.9800,...,2.410000,2.215000,1.526667,1.156667,0.400000,0.205000,0.020,1,1,0.845754
229,9020,-18.0,18.0,96.16,234.27,-78.43,-22.140000,-22.273333,-22.235,-22.3500,...,2.250000,2.260000,2.250000,2.055000,1.366667,0.996667,0.240,1,1,0.841252
234,9025,-18.0,18.0,96.70,234.81,-77.89,-21.820000,-21.600000,-21.820,-21.8900,...,1.906667,1.536667,0.780000,0.585000,0.400000,0.380000,0.390,1,1,0.839947
233,9024,-18.0,18.0,96.31,234.42,-78.28,-22.200000,-22.210000,-21.990,-22.2100,...,2.205000,1.516667,1.146667,0.390000,0.195000,0.010000,-0.010,1,1,0.839898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,9067,-18.0,18.0,97.16,235.27,-77.43,-25.700000,-25.750000,-25.725,-25.3500,...,0.765000,0.620000,0.590000,0.530000,0.460000,0.020000,0.010,1,1,0.753862
307,9098,-18.0,18.0,96.82,234.93,-77.77,-20.896667,-20.750000,-20.630,-20.5100,...,-0.690000,-0.670000,-0.560000,-0.640000,-0.590000,-0.590000,-0.265,1,1,0.753534
309,9100,-18.0,18.0,96.73,234.84,-77.86,-20.840000,-20.720000,-20.600,-21.7625,...,-0.760000,-0.650000,-0.730000,-0.680000,-0.680000,-0.355000,-0.140,1,1,0.753409
305,9096,-18.0,18.0,97.19,235.30,-77.40,-20.995000,-20.526667,-20.380,-20.2600,...,-0.355000,-0.320000,-0.300000,-0.190000,-0.270000,-0.220000,-0.220,1,1,0.752354


In [95]:
# Importing the metrics package from sklearn library
from sklearn import metrics
# Creating the confusion matrix
cm = metrics.confusion_matrix(most_certain[605], most_certain['predicted_labels'])
# Assigning columns names
cm_df = pd.DataFrame(cm, 
            columns = ['Predicted Negative', 'Predicted Positive'],
            index = ['Actual Negative', 'Actual Positive'])
# Showing the confusion matrix
cm_df

,Predicted Negative,Predicted Positive
Actual Negative,0,16
Actual Positive,0,106


In [ ]:
most_certain.query('predicted_labels==1').describe()

In [ ]:
most_certain.query('predicted_labels==0').describe()